In [21]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


In [22]:
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - \
        np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360-angle

    return angle


In [23]:
import tkinter as tk
import cv2
from PIL import Image, ImageTk
import mediapipe as mp
import numpy as np

# Create the main window
window = tk.Tk()

# Get the screen width and height
screen_width = window.winfo_screenwidth()
screen_height = window.winfo_screenheight()

# Set the window size to full screen
window.geometry(f"{screen_width}x{screen_height}")


# Create the heading rectangle frame
heading_frame = tk.Frame(window, bg="gray", width=screen_width, height=100)
heading_frame.place(x=0, y=0)

# Add text to the heading frame
heading_label = tk.Label(heading_frame, text="Welcome Dikshit", font=(
    "Arial", 24), fg="white", bg="gray")
heading_label.place(relx=0.5, rely=0.5, anchor="center")

# Create the first section frame
frame1 = tk.Frame(window, bg="red", width=(
    screen_width*2)/3, height=(screen_height*2)/3)
frame1.place(x=0, y=100)

# Create the second section frame
frame2 = tk.Frame(window, bg="blue", width=screen_width /
                  3, height=(screen_height*2)/3)
frame2.place(x=(screen_width*2)//3, y=100)

# Create the third section frame
frame3 = tk.Frame(window, bg="green", width=(
    screen_width*2)/3, height=screen_height//3)
frame3.place(x=0, y=100 + (screen_height*2)/3)

# Create the fourth section frame
frame4 = tk.Frame(window, bg="yellow", width=screen_width //
                  3, height=screen_height//3)
frame4.place(x=(screen_width*2)/3, y=100 + (screen_height*2)/3)
# Open a video capture
cap = cv2.VideoCapture(0)

# Function to update video frames


def update_video():
    global frame1, cap, counter, stage

    ret, frame = cap.read()
    frame = cv2.resize(frame, (frame1.winfo_width(), frame1.winfo_height()))

    # Recolor image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # Make detection
    results = pose.process(image)

    # Recolor back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Extract landmarks
    try:
        landmarks = results.pose_landmarks.landmark

        # Get coordinates
        shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

        # Calculate angle
        angle = calculate_angle(shoulder, elbow, wrist)

        # Visualize angle
        cv2.putText(image, str(angle),
                    tuple(np.multiply(elbow, [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,
                                                    255, 255), 2, cv2.LINE_AA
                    )

        # Curl counter logic
        if angle > 160:
            stage = "down"
        if angle < 30 and stage == 'down':
            stage = "up"
            counter += 1
            print(counter)

    except:
        pass

    # Render curl counter
    # Setup status box
    cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)

    # Rep data
    cv2.putText(image, 'REPS', (15, 12),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, str(counter),
                (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

    # Stage data
    cv2.putText(image, 'STAGE', (65, 12),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, stage,
                (60, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

    # Convert the image to PIL format
    img = Image.fromarray(image)

    # Create PhotoImage object to display in the label
    img_tk = ImageTk.PhotoImage(img)

    # Update the label with the new image
    label.config(image=img_tk)
    label.image = img_tk

    # Schedule the next update
    if window.winfo_exists():
        frame1.after(10, update_video)
    else:
        cap.release()


# Create an instance of mediapipe pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Add label to display video frames
label = tk.Label(frame1)
label.place(x=0, y=0, relwidth=1, relheight=1)
# Add labels to the other frames
label2 = tk.Label(frame2, text="Section 2", fg="white", bg="green")
label2.pack()

label3 = tk.Label(frame3, text="Section 3", fg="white", bg="blue")
label3.pack()

label4 = tk.Label(frame4, text="Section 4", fg="black", bg="yellow")
label4.pack()
# Initialize counter and stage
counter = 0
stage = None

# Start updating video frames
update_video()

# Bind the camera release to window closing event


def release_camera():
    if cap.isOpened():
        cap.release()
    window.destroy()


window.protocol("WM_DELETE_WINDOW", release_camera)

# Start the Tkinter event loop
window.mainloop()


1
